# Imports

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
from numba import njit
from itertools import product

# Sample Data

## Load Data

In [2]:
data_org = pd.read_csv('../data/family_data.csv', index_col='family_id')

## Extractioin

In [3]:
data = data_org.iloc[0:200,:]

In [4]:
mean_occupnacy = data.n_people.sum()/len(data)
print('MEAN OCCUPANCY : ', mean_occupnacy)

MEAN OCCUPANCY :  4.1


In [5]:
data.shape

(200, 11)

## Set

In [6]:
N_DAYS = 100
N_FAMILIES = len(data)
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

FAMILY_SIZE = data.n_people.values
DESIRED     = data.values[:, :-1] - 1

In [8]:
GAP = range(MIN_OCCUPANCY,MAX_OCCUPANCY+1)
GAP_P = {}
_C_GAP =30
for i in GAP:
    GAP_P[i]= (range(max(i-_C_GAP,MIN_OCCUPANCY),min(i+_C_GAP,MAX_OCCUPANCY)))

# Cost Function

## Preference Cost

### Preference Cost Function

In [9]:
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9 * n
    elif choice == 3:
        penalty = 100 + 9 * n
    elif choice == 4:
        penalty = 200 + 9 * n
    elif choice == 5:
        penalty = 200 + 18 * n
    elif choice == 6:
        penalty = 300 + 18 * n
    elif choice == 7:
        penalty = 300 + 36 * n
    elif choice == 8:
        penalty = 400 + 36 * n
    elif choice == 9:
        penalty = 500 + 36 * n + 199 * n
    else:
        penalty = 500 + 36 * n + 398 * n
    return penalty

In [10]:
def GetPreferenceCostMatrix(data):
    cost_matrix = np.zeros((N_FAMILIES, N_DAYS), dtype=np.int64)
    for i in range(N_FAMILIES):
        desired = data.values[i, :-1]
        cost_matrix[i, :] = get_penalty(FAMILY_SIZE[i], 10)
        for j, day in enumerate(desired):
            cost_matrix[i, day-1] = get_penalty(FAMILY_SIZE[i], j)
    return cost_matrix

### Make Matrix

In [11]:
PCOSTM = GetPreferenceCostMatrix(data)

In [12]:
print(PCOSTM.shape)
PCOSTM[0]

(200, 100)


array([2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236,  544, 2236,
         86, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236,
       2236, 2236, 2236, 2236, 2236, 1440, 2236, 2236, 2236, 2236,  236,
       2236, 2236, 2236, 2236,   50, 2236, 2236, 2236, 2236, 2236, 2236,
       2236, 2236, 2236, 2236, 2236, 2236, 2236,    0, 2236, 2236, 2236,
       2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236,  372, 2236, 2236,
       2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236,  272,  444, 2236,
       2236, 2236, 2236, 2236,  136, 2236, 2236, 2236, 2236, 2236, 2236,
       2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236, 2236,
       2236])

### Calculate Preference Cost

In [13]:
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i, p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty += PCOSTM[i, p]
        daily_occupancy[p] += n
    return penalty, daily_occupancy

## Accounting Cost

In [14]:
def GetAccountingCostMatrix():
    ac = np.zeros((MAX_OCCUPANCY+1, MAX_OCCUPANCY+1), dtype=np.float64)
    for n in range(ac.shape[0]):
        for n_p1 in range(ac.shape[1]):
            diff = abs(n - n_p1)
            ac[n, n_p1] = max(0, (n - MIN_OCCUPANCY) / 400 * n**(0.5 + diff / 50.0))
#             ac[n, n_p1] = max(0, (n - 2) / 8 * n**(0.5 + diff))
    return ac

### Make Matirx

In [15]:
ACOSTM = GetAccountingCostMatrix() 

In [16]:
print(ACOSTM.shape)
ACOSTM[8]

(301, 301)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

### Accounting Cost

In [17]:
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    n_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day + 1]
        n    = daily_occupancy[day]
        n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        accounting_cost += ACOSTM[n, n_p1]
    return accounting_cost, n_out_of_range

## Total Cost

In [18]:
@njit(fastmath=True)
def cost_function(prediction):
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, n_out_of_range = acost(daily_occupancy)
    return penalty + accounting_cost + n_out_of_range*100000000

## Evaluation Method

In [19]:
def eval(prediction):
    pc, occ = pcost(prediction)
    ac, _ = acost(occ)
    print('Preferenced Cost : ', pc)
    print('Accounting Cost : {: .2f}'.format(ac))
    print('Total Cost : {: .2f}'.format(pc+ac))
    print('')
    print('Max Occupancy : {} , Min Ocupancy : {}'.format(occ.max(), occ.min()))

# Minimize Practice

## Solver

In [20]:
def IP():
    
    S = pywraplp.Solver('SolveAssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#     S = pywraplp.Solver('SolveAssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)    
#     GAP = MAX_OCCUPANCY-MIN_OCCUPANCY + 1
    
    candidates = [[] for _ in range(N_DAYS)] 
    
    x = {}
    for i in range(N_FAMILIES):
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            x[i, j] = S.BoolVar('x[%i,%i]' % (i, j))
    
    N = {}
    for day in range(N_DAYS):
        for i in GAP:
            for j in GAP_P[i]:
                N[day, i, j] = S.BoolVar('N[%i,%i,%i]' % (day, i, j))
            

    family_presence = [S.Sum([x[i, j] for j in DESIRED[i, :]])
                                                        for i in range(N_FAMILIES)]
    
    linear_constraint = [S.Sum(N[day, i, j] for i in GAP
                                                                                                                               for j in GAP_P[i]) 
                                                                                                                               for day in range(N_DAYS) ]
    
    daily_occupancy_x = [S.Sum([x[i, j] * FAMILY_SIZE[i] for i in candidates[j]])
                                                                                      for j in range(N_DAYS)]
    
#     daily_occupancy_n = [S.Sum( [(i)* N[day, i, j] for i in GAP
#                                                                                                                                                                            for j in GAP_P[i]] )
#                                                                                                                                                                            for day in range(N_DAYS)]
    
 
        
    # Objective    
    partial_costs = [PCOSTM[i, j] * x[i,j] for i in range(N_FAMILIES) for j in DESIRED[i, :] ]
    preference_cost = S.Sum(partial_costs)
    
    penalties = []
    for day in range(N_DAYS):
        for i in GAP:
            for j in GAP_P[i]:
                daily_occupancy = i
                p_occupancy = j
                penalties.append(abs(daily_occupancy - MIN_OCCUPANCY)/400 * daily_occupancy**(1/2+abs(daily_occupancy - p_occupancy)/50) * N[day, daily_occupancy, p_occupancy])

    accounting_penalty = S.Sum(penalties)
    
    total_cost = S.Sum([preference_cost,accounting_penalty])
#     total_cost = preference_cost + accounting_penalty

    S.Minimize(total_cost)
#     S.Minimize(accounting_penalty)
#     S.Minimize(preference_cost)


# 
    # Constraints
    for day in range(N_DAYS):
        S.Add(linear_constraint[day] == 1)
        S.Add(
#         daily_occupancy_x[day] - daily_occupancy_n[day] == 0
            S.Sum([N[day, i, j]*(i) for i in GAP for j in GAP_P[i]]) 
                == daily_occupancy_x[day]
        )       

    for day in range(N_DAYS-1):
        S.Add(
            S.Sum([N[day, i, j]*(j) for i in GAP for j in GAP_P[i]]) 
                == daily_occupancy_x[day+1]
        ) 
        
#         S.Add(daily_occupancy_n[day] == 0)
        
    for i in range(N_FAMILIES):
        S.Add(family_presence[i] == 1)
        
# #         差の条件を加えてみる
#     for j in range(N_DAYS-1):
#         S.Add(daily_occupancy[j]   - daily_occupancy[j+1] <= 23)
#         S.Add(daily_occupancy[j+1] - daily_occupancy[j]   <= 23)
            
        

    print('ready !!')
    S.EnableOutput() 
    S.SetNumThreads(24)
#     S.SuppressOutput()
#     S.SetTimeLimit(1000*60*10)
    res = S.Solve()

    resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
               4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}

    print('IP solver result:', resdict[res])


    l = [(i, j, x[i, j].solution_value()) for i in range(N_FAMILIES)
                                                      for j in DESIRED[i, :] 
                                                      if x[i, j].solution_value()>0]
    

    df = pd.DataFrame(l, columns=['family_id', 'day', 'n'])
    return df

In [21]:
GAP

range(125, 301)

In [22]:
def make_IP():
    S = pywraplp.Solver('SolveAssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#     S = pywraplp.Solver('SolveAssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)    
#     GAP = MAX_OCCUPANCY-MIN_OCCUPANCY + 1
    
    candidates = [[] for _ in range(N_DAYS)] 
    
    x = {}
    for i in range(N_FAMILIES):
        for j in DESIRED[i, :]:
            candidates[j].append(i)
            x[i, j] = S.BoolVar('x[%i,%i]' % (i, j))
    
    N = {}
    for day in range(N_DAYS):
        for i in GAP: 
            for j in GAP_P[i]:
                N[day, i, j] = S.BoolVar('N[%i,%i,%i]' % (day, i, j))
            

    family_presence = [S.Sum([x[i, j] for j in DESIRED[i, :]])
                                                        for i in range(N_FAMILIES)]
    
    linear_constraint = [S.Sum(N[day, i, j] for i in GAP
                                                                                                                               for j in GAP_P[i]) 
                                                                                                                               for day in range(N_DAYS) ]
    
    daily_occupancy_x = [S.Sum([x[i, j] * FAMILY_SIZE[i] for i in candidates[j]])
                                                                                      for j in range(N_DAYS)]
    
#     daily_occupancy_n = [S.Sum( [(i)* N[day, i, j] for i in GAP
#                                                                                                                                                                            for j in GAP_P[i]] )
#                                                                                                                                                                            for day in range(N_DAYS)]
    
 
        
    # Objective    
    partial_costs = [PCOSTM[i, j] * x[i,j] for i in range(N_FAMILIES) for j in DESIRED[i, :] ]
    preference_cost = S.Sum(partial_costs)
    
    penalties = []
    for day in range(N_DAYS):
        for i in GAP:
            for j in GAP_P[i]:
                daily_occupancy = i
                p_occupancy = j
                penalties.append(abs(daily_occupancy - MIN_OCCUPANCY)/400 * daily_occupancy**(1/2+abs(daily_occupancy - p_occupancy)/50) * N[day, daily_occupancy, p_occupancy])

    accounting_penalty = S.Sum(penalties)
    
    total_cost = S.Sum([preference_cost,accounting_penalty])
#     total_cost = preference_cost + accounting_penalty

    S.Minimize(total_cost)
#     S.Minimize(accounting_penalty)
#     S.Minimize(preference_cost)


# 
    # Constraints
    for day in range(N_DAYS):
        S.Add(linear_constraint[day] == 1)
        S.Add(
#         daily_occupancy_x[day] - daily_occupancy_n[day] == 0
            S.Sum([N[day, i, j]*(i) for i in GAP for j in GAP_P[i]]) 
                == daily_occupancy_x[day]
        )       

    for day in range(N_DAYS-1):
        S.Add(
            S.Sum([N[day, i, j]*(j) for i in GAP for j in GAP_P[i]]) 
                == daily_occupancy_x[day+1]
        ) 
        
#         S.Add(daily_occupancy_n[day] == 0)
        
    for i in range(N_FAMILIES):
        S.Add(family_presence[i] == 1)
        
#     for day in range(N_DAYS):
#         for i in range(MIN_OCCUPANCY):
#             for j in range(MIN_OCCUPANCY):
#                 if abs(i - j)
    
        
# #         差の条件を加えてみる
#     for j in range(N_DAYS-1):
#         S.Add(daily_occupancy[j]   - daily_occupancy[j+1] <= 23)
#         S.Add(daily_occupancy[j+1] - daily_occupancy[j]   <= 23)
            
        

    print('ready !!')
    S.EnableOutput() 
#     S.SetNumThreads(24)
#     S.SuppressOutput()
#     S.SetTimeLimit(1000*60*10)

    return S
    res = S.Solve()

In [23]:
S = make_IP()

ready !!


In [24]:
lp = S.ExportModelAsLpFormat(False)

In [25]:
with open('santa_sm.lp', 'w') as f:
  print(lp, file=f)

In [26]:
# res = S.Solve()
# resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
#            4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}

# print('IP solver result:', resdict[res])


# l = [(i, j, x[i, j].solution_value()) for i in range(N_FAMILIES)
#                                                   for j in DESIRED[i, :] 
#                                                   if x[i, j].solution_value()>0]


# df = pd.DataFrame(l, columns=['family_id', 'day', 'n'])


## Solve

In [294]:
%%time
df_tmp = IP()

KeyboardInterrupt: 

## Check

In [183]:
df_tmp.shape

(110, 3)

In [184]:
eval(df_tmp.day.values)

Preferenced Cost :  50
Accounting Cost :  7.49
Total Cost :  57.49

Max Occupancy : 27 , Min Ocupancy : 0


In [152]:
pc, occ = pcost(df_tmp.day.values)
print(occ)

[29  2 15  0 11  0  0  0  6  7  6 12  5  0  5  3 13 10 10  4  0  2  2  7
 13  4  0  2  3  0 27 18  4  9  0  0  4  4 19  0  0  0  0  0  6 10 20  0
  2  0  0 15 10 11  0  5  4  2  0  0  6  0  0  0  0  3  6 10  0  0  0  0
  0  7 15  0  0  0  0  0 13  4  0  0  2  0  0  4 11  0  3  0  0  3  0 11
  0  0  0  3  0]


In [198]:
ans = df_tmp.day.values

In [200]:
ans[69]

4

In [190]:
print(df_tmp.day.values)

[51 25 99  1 52 31 87 24 17  0 95 18 51 53 44 21 45 46 74  2  2 55 60 18
 74 15 57 37 80 88 67 73 23 31 45 30 46 14 12  9 24 30 80 10 48  4 14 44
 31 32 66 27 32 36 93 90 11 30 73 38  0 17  0 30  2 46 65  0 88  4 31 10
 65  9 81 52 45 19 51  0 17 16 31 87  8 53 28 24 56 22 53 16 84  4 46 23
 74 95  2 38 38 52 30 67 46 11 33 11 33 16]
